# Thinkful Capstone Project: "Etsy Epiphany"
### By Tiani Calip

# Problem Addressed

## Personal Interest and Problem
My mom owns an Etsy online store and is constantly stressing out about how well her products are doing, what price she should be listing her products at, and what product she should make and sell next. I want to create a solution for her to be able to identify what the most popular products are, how the reviews trend over time, and how the sentiment of review text maps against the review ratings. This way, she would be able to see how her products are doing compared to similar items on Etsy, and be able to respond to the demand for certain types of popular items and create and sell her own. Sometimes, she doesn’t understand why one shirt sells more than another. Providing review mapping and sentiment analysis will allow her and other users to identify what qualities of their products are favorable or not, be able to address those during production, and thereby possibly increase their chance of getting favorited.  

## Goal  
Using NLP and machine learning techniques, I will identify the top 20 important qualities (description, title, and tags) needed for any given Etsy user to use to improve their chance of getting favorited.  

## Hypothesis
By using machine learning and NLP, I can give suggesthions on product descriptions and help a store increase their favorites by 5%.

### Gameplan:   
Get the shop info for every shop that can be found with the keyword: "Disney"  
Use shop_ids to get the listing info   
Use the value of the feedback to train a classification model to know when a sentiment is positive or negative  
Identify the most important digrams/phrases in a shop's collective messages that lead to a positive sentiment  
Compare prices of the listings that have those positive sentiments  
Compare # of favorites to price of the listings with the positive sentiments  
Compare that with my mom's sentiments and prices and give suggestions for price and product qualities


fork out positive reviews and negative. see what ppl are saying abt both, give advice to the stores for what they could change abt their stores and improve.  
give suggestions abt what shop can do that others/competetors are doing  


## Impact  
I can determine the important qualities of a product that lead to the best reviews and review sentiments, then identify the most optimal price to increase the number of favorites. This can help my mom and any other prospective users to get ideas for their next products, and have confidence to list the products at an optimal price for both the seller and the buyer.  

# Dataset  
Data source = Etsy itself! I used the Etsy API to collect the data in real time from shops that have the keyword "Disney"

## Existing Efforts
No existing Etsy apps look at both review scores and sentiments in an effort to identofy important qualities. I'm excited to see if I can be the first!  

# Project Concerns


## Assumptions
I am making the assumption that the limits and offset that I chose for my API requests will give me a good representation of the Disney themed Etsy store population

## Risks (of data or models)
This is my first time using an API outside of the Thinkful course. I do not expect it to be an easy learning curve.  
Some data that would be ideal requires an OAuthentification, which means any user I want to scrape data from needs to verify and approve my data scraping. I do not want random users to feel threatened by my data grabbing, so I will have to go in a roundabout way to grab the data I want.  

## Questions Resulting
What is the best way to measure my scenario (binary outcome, probability outcome, or quantitative outcome)?


# Outcomes
To finish later

## Future work:
Look at the number of sales of those listing, which requires shop owner access and authentication to grab that data.
Compare sales to price of the listings with the positive sentiments.  
See how changing the descriptions, titles, tags, and prices can increase favorites and sales.  

In [1]:
import pandas as pd
import numpy as np

In [37]:
# Open all the csv files containing the data frames from my Etsy API requests
# Listing id, user id (person who favorite'd)
num_favs_agg = pd.read_csv('num_favs_agg.csv')
# shop, listing, tag, description, price, user_id is shop owner
df_listings = pd.read_csv('dis_shop_listings_df.csv')
listing_tsz_df = pd.read_csv('listing_tsz_df.csv')
df_shops = pd.read_csv('disney_shopssdf.csv')  # Number of listings shop
df_creation_tsz = pd.read_csv('df_creation_tsz.csv')
# user_id is id of shop owner
df_reviews = pd.read_csv('dis_shop_reviewsdf.csv')

# Clean data
df_reviews['message'] = df_reviews['message'].fillna('')
df_reviews['message'] = df_reviews['message'].apply(
    lambda x: x.replace(str('&#39;'), str('\'')).replace(str('\r'), str('')).replace(str('\n'), str('')).replace(str('&quot;'), str('')))
df_listings['title'] = df_listings['title'].apply(
    lambda x: x.replace(str('&#39;'), str('\'')).replace(str('&quot;'), str('')))
df_listings['description'] = df_listings['description'].apply(
    lambda x: x.replace(str('&#39;'), str('\'')).replace(str('\r'), str('')).replace(str('\n'), str('')).replace(str('&quot;'), str('')))

Way #1: Shop level aggregation
Each row is a shop . . . all listings are smashed together, all reviews are smashed together

In [39]:
# Join all the listing titles per shop
title_all = df_listings.groupby('user_id')['title'].apply(
    lambda x: " ".join(x)).reset_index()
# Join all the listing descriptions per shop
description_all = df_listings.groupby('user_id')['description'].apply(
    lambda x: " ".join(x)).reset_index()
# Join all the listing tags per shop
tags_all = df_listings.groupby('user_id')['tags'].apply(lambda x: " ".join(
    x).replace('[', '').replace(']', '').replace("'", '').lower()).reset_index()
# Join all the reviews per shop
reviews = df_reviews.groupby('user_id')['message'].apply(
    lambda x: " ".join(x.fillna(''))).reset_index()
# Average all the prices to get average price of shop
average_price_per_shop = df_listings.groupby(
    'user_id')['price'].agg('mean').reset_index()
# Average all the values to get average value of shop
value = df_reviews.groupby('user_id')['value'].agg('mean').reset_index()

In [40]:
# Merge all the combined data I made
shop_description_df = pd.merge(pd.merge(
    pd.merge(title_all, description_all), tags_all), average_price_per_shop)
# Merge the two review data together
df_rev = pd.merge(reviews, value)

In [41]:
# Make a dataframe with shops and listing info
df_shops2 = pd.merge(df_shops.drop_duplicates(), df_listings, on='user_id')
print(df_shops.shape)
print(df_listings.shape)
print(df_shops2.shape)

(200, 6)
(7370, 10)
(7370, 15)


In [42]:
# Make a dataframe with shops and listing info
df_shops2 = pd.merge(df_shops.drop_duplicates(), df_listings, on='user_id')
df_shops2.shape

(7370, 15)

In [43]:
num_favs_agg = num_favs_agg.drop('index', axis=1)

In [44]:
num_favs_agg.groupby('listing_id').agg('count').max()

num_favorites    3
dtype: int64

In [45]:
# df_favs_agg = df_favs_agg.drop_duplicates(subset=['listing_id'])
num_favs_agg = num_favs_agg.drop_duplicates(subset=['listing_id'])
# Merge shop and listing info with the fav info
df_shops2 = pd.merge(df_shops2, num_favs_agg, how='left')
df_shops2.shape

(7370, 16)

In [46]:
df_shops2['num_favorites'] = df_shops2['num_favorites'].fillna(0)

In [47]:
df_shops2.num_favorites.unique()

array([ 39.,  24.,   0.,   6.,   4.,  36.,  38.,  37.,  31.,  42.,  34.,
        29.,  13.,   7.,  15.,   2.,   9.,   1.,  30.,  23.,   8.,  14.,
        25.,  18.,  20.,   3.,   5.,  12.,  16., 100.,  11.,  10.,  32.,
        28.,  57.,  49.,  48.,  21.,  17.,  27.,  41.,  59.,  62.,  46.,
        55.,  19.,  22.,  47.,  70.,  44.,  84.,  33.,  71.,  40.,  79.,
        26.,  35.,  52.,  61.,  77.,  80.,  58.,  88.,  82.,  76.,  99.,
        45.,  85.,  68.,  56.,  73.,  50.,  94.,  86.,  51.,  43.,  64.,
        98.,  67.,  93.,  75.,  78.,  65.,  72.,  89.,  66.,  54.,  63.,
        60.,  95.,  74.,  81.,  92.,  53.,  97.,  90.,  83.,  91.,  69.,
        87., 104., 102., 150., 112.,  96., 118., 134., 106., 144.])

In [48]:
df_shops2 = df_shops2[['listing_id', 'shop_name', 'user_id', 'num_favorites']].drop_duplicates()

In [49]:
df_shops2[['listing_id', 'shop_name', 'user_id', 'num_favorites']].groupby('listing_id').agg('count').max()

shop_name        1
user_id          1
num_favorites    1
dtype: int64

In [50]:
# Get the # of favs per listing for each shop
df_shops_fav_agg = df_shops2.groupby(['user_id', 'shop_name'])[
    'num_favorites'].agg('sum').reset_index()
df_shops_fav_agg.shape

(198, 3)

In [51]:
df_shops2

,listing_id,shop_name,user_id,num_favorites
0,492259062,DisneyObscura,20413,39.0
1,464863490,DisneyObscura,20413,24.0
2,243657597,DisneyObscura,20413,0.0
3,625330096,DisneyObscura,20413,0.0
4,639116667,DisneyObscura,20413,6.0
5,625280804,DisneyObscura,20413,0.0
6,639107951,DisneyObscura,20413,4.0
7,243521750,DisneyObscura,20413,36.0
8,243518492,DisneyObscura,20413,38.0
9,241680506,DisneyObscura,20413,37.0


In [52]:
df_shops_fav_agg

,user_id,shop_name,num_favorites
0,20413,DisneyObscura,355.0
1,6112479,DisneyBottleCaps,1113.0
2,7543374,DisneyGirl84,12.0
3,8082338,disneyfarmhouse,0.0
4,11738325,DisneyDivas,26.0
5,18713976,NataliFashionDOLL,121.0
6,19856189,DisneyBlueLine,153.0
7,20850951,MakingCruiseMemories,2652.0
8,20986912,disneyaddictedears,0.0
9,21590497,AlicesPlants,551.0


In [53]:
# Merge all the info together
df_shop_final = pd.merge(df_shops_fav_agg, shop_description_df)
print(df_shop_final.shape)
df_shop_final = pd.merge(df_shop_final, df_rev)
print(df_shop_final.shape)
# Get the # of active listings for each shop
num_listings = pd.DataFrame(df_shops[['shop_name', 'listing_active_count']])
df_shop_final = pd.merge(df_shop_final, num_listings)
print(df_shop_final.shape)
# Add the creation time in epoch seconds
df_shop_final = pd.merge(df_shop_final, df_creation_tsz)
print(df_shop_final.shape)

(198, 7)
(131, 9)
(132, 10)
(134, 11)


In [54]:
num_favs_agg['listing_id'].drop_duplicates().shape

(5888,)

In [55]:
num_favs_agg['listing_id'].shape

(5888,)

In [56]:
df_shop_final

,user_id,shop_name,num_favorites,title,description,tags,price,message,value,listing_active_count,creation_tsz
0,20413,DisneyObscura,355.0,Moana toddler baby with Stitch crossover mashu...,Baby Moana making friends with Stitch on the b...,"moana, baby moana, toddler moana, stitch, beac...",57.277778,"if i could give this 1,000 stars i would!! i l...",1.000000,18,1139103736
1,6112479,DisneyBottleCaps,1113.0,Disney Princess POCAHONTAS 3D Finished Bottle ...,♥ Finished Bottle Cap ♥Finished Bottle Caps fo...,"disney, disney princess, princess pocahontas, ...",7.759690,Perfect party favor for my daughter’s birthday...,0.992278,226,1423070214
2,7543374,DisneyGirl84,12.0,Princess Leia custom disney ears,These cute ears are inspired by Star Wars and ...,"gifts, disney, vacation, disneyland, star wars...",20.000000,Great buyer and easy to communicate with Grea...,1.000000,1,1293941621
3,8082338,disneyfarmhouse,0.0,Disney autograph signing board Disney autograp...,"Disneyland, Walt Disney World, or Disney cruis...",,15.000000,"I was expecting a canvas, and got a night piec...",1.000000,5,1517611905
4,11738325,DisneyDivas,26.0,Personalized superhero t shirt Personalized Ba...,Personalized super hero t shirt with super kid...,"super hero, marvel, spiderman, superman, batma...",14.400000,Came a little late but is excellent work.,1.000000,5,1378661204
5,18713976,NataliFashionDOLL,121.0,Clothing For Dolls DisneyAnimators Clothing Fo...,"Disney Animator, suitable for all dolls in thi...","clothes for dolls, dress, doll sandals, 50cm...",18.032258,Purchased two dresses and they are both AMAZIN...,1.000000,31,1522249270
6,19856189,DisneyBlueLine,153.0,Thank you for your service coin Thin Blue Line...,This is a Fantasy coin 2.5 inches cut I to a M...,,9.615385,My friend loved her birthday present. Thank yo...,1.000000,13,1470967396
7,20850951,MakingCruiseMemories,2652.0,"Disney Cruise Door Magnets Disney Shirt Man, I...",**NEW LARGER SIZE LITTLE HEADS**You will recei...,"disney cruise, disney magnet, cruise magnet, d...",14.485054,"My daughter loves this shirt, fast delivery! B...",0.987500,184,1380657794
8,20986912,disneyaddictedears,0.0,Marie! Deadpool! Animal print Frozen! Pink lad...,Cuz I'm a lady! Lovely pair of Marie ears! Per...,"marie marvel, deadpool animal print anna and e...",23.459459,I love my new ears!!! Going to Disney in Decem...,1.000000,37,1453754312
9,21590497,AlicesPlants,551.0,Cactus Seed Mix Grow Your Own Kit Barrell Cact...,A Cactus Seed Mix grow your own plant kit. Her...,"cactus, grow your own kit, plant kit, seed kit...",5.781905,"Looks great and is very secure, thanks!!! love...",1.000000,21,1386189807


In [57]:
# Drop the rows that have shops without Disney-themed items
df_shop_final = df_shop_final[df_shop_final['user_id'] != 37407286]
df_shop_final = df_shop_final[df_shop_final['user_id'] != 82117396]
df_shop_final = df_shop_final[df_shop_final['user_id'] != 120126609]
df_shop_final = df_shop_final[df_shop_final['user_id'] != 21590497]

In [58]:
df_shop_final = df_shop_final.drop_duplicates()

In [59]:
df_shop_final.head()

,user_id,shop_name,num_favorites,title,description,tags,price,message,value,listing_active_count,creation_tsz
0,20413,DisneyObscura,355.0,Moana toddler baby with Stitch crossover mashu...,Baby Moana making friends with Stitch on the b...,"moana, baby moana, toddler moana, stitch, beac...",57.277778,"if i could give this 1,000 stars i would!! i l...",1.000000,18,1139103736
1,6112479,DisneyBottleCaps,1113.0,Disney Princess POCAHONTAS 3D Finished Bottle ...,♥ Finished Bottle Cap ♥Finished Bottle Caps fo...,"disney, disney princess, princess pocahontas, ...",7.759690,Perfect party favor for my daughter’s birthday...,0.992278,226,1423070214
2,7543374,DisneyGirl84,12.0,Princess Leia custom disney ears,These cute ears are inspired by Star Wars and ...,"gifts, disney, vacation, disneyland, star wars...",20.000000,Great buyer and easy to communicate with Grea...,1.000000,1,1293941621
3,8082338,disneyfarmhouse,0.0,Disney autograph signing board Disney autograp...,"Disneyland, Walt Disney World, or Disney cruis...",,15.000000,"I was expecting a canvas, and got a night piec...",1.000000,5,1517611905
4,11738325,DisneyDivas,26.0,Personalized superhero t shirt Personalized Ba...,Personalized super hero t shirt with super kid...,"super hero, marvel, spiderman, superman, batma...",14.400000,Came a little late but is excellent work.,1.000000,5,1378661204


In [60]:
df_shop_final['listing_active_count'].unique()

array([  18,  226,    1,    5,   31,   13,  184,   37,   16,   14,   23,
         53,   10,    8,   95,   34,    4,   24,  711,   50,    9,  115,
         40,    6,    7,   46,    2,  141,   32,  207,  212,  171,  122,
       1132,  103,  829,  371,   36,   11,   30,   21,    3,   17,   52,
         51,   69,   19,   39,   26,   48,   15,  117,  106,   41,  116,
        123,   22,  177,   12,   47,  236,   54,   42,   57,   73,  254,
         33,   29,   99], dtype=int64)

In [61]:
# Add year of shop creation
df_shop_final['creation_tsz'] = pd.to_datetime(
    df_shop_final['creation_tsz'], unit='s')
df_shop_final['creation_tsz'] = df_shop_final['creation_tsz'].dt.year
df_shop_final = df_shop_final.rename(columns={'creation_tsz': 'creation_yr'})
df_shop_final

,user_id,shop_name,num_favorites,title,description,tags,price,message,value,listing_active_count,creation_yr
0,20413,DisneyObscura,355.0,Moana toddler baby with Stitch crossover mashu...,Baby Moana making friends with Stitch on the b...,"moana, baby moana, toddler moana, stitch, beac...",57.277778,"if i could give this 1,000 stars i would!! i l...",1.000000,18,2006
1,6112479,DisneyBottleCaps,1113.0,Disney Princess POCAHONTAS 3D Finished Bottle ...,♥ Finished Bottle Cap ♥Finished Bottle Caps fo...,"disney, disney princess, princess pocahontas, ...",7.759690,Perfect party favor for my daughter’s birthday...,0.992278,226,2015
2,7543374,DisneyGirl84,12.0,Princess Leia custom disney ears,These cute ears are inspired by Star Wars and ...,"gifts, disney, vacation, disneyland, star wars...",20.000000,Great buyer and easy to communicate with Grea...,1.000000,1,2011
3,8082338,disneyfarmhouse,0.0,Disney autograph signing board Disney autograp...,"Disneyland, Walt Disney World, or Disney cruis...",,15.000000,"I was expecting a canvas, and got a night piec...",1.000000,5,2018
4,11738325,DisneyDivas,26.0,Personalized superhero t shirt Personalized Ba...,Personalized super hero t shirt with super kid...,"super hero, marvel, spiderman, superman, batma...",14.400000,Came a little late but is excellent work.,1.000000,5,2013
5,18713976,NataliFashionDOLL,121.0,Clothing For Dolls DisneyAnimators Clothing Fo...,"Disney Animator, suitable for all dolls in thi...","clothes for dolls, dress, doll sandals, 50cm...",18.032258,Purchased two dresses and they are both AMAZIN...,1.000000,31,2018
6,19856189,DisneyBlueLine,153.0,Thank you for your service coin Thin Blue Line...,This is a Fantasy coin 2.5 inches cut I to a M...,,9.615385,My friend loved her birthday present. Thank yo...,1.000000,13,2016
7,20850951,MakingCruiseMemories,2652.0,"Disney Cruise Door Magnets Disney Shirt Man, I...",**NEW LARGER SIZE LITTLE HEADS**You will recei...,"disney cruise, disney magnet, cruise magnet, d...",14.485054,"My daughter loves this shirt, fast delivery! B...",0.987500,184,2013
8,20986912,disneyaddictedears,0.0,Marie! Deadpool! Animal print Frozen! Pink lad...,Cuz I'm a lady! Lovely pair of Marie ears! Per...,"marie marvel, deadpool animal print anna and e...",23.459459,I love my new ears!!! Going to Disney in Decem...,1.000000,37,2016
10,21726696,DesignedMemories46,288.0,Minnie Mouse Autograph & Scrapbook Minnie Mous...,Trying to capture those perfect Disney moments...,"disney photo album, signature books, disneylan...",34.375000,Beautiful album and amazing communication from...,1.000000,16,2015


In [62]:
# Save as a csv file for later use
df_shop_final = df_shop_final.to_csv('final_df.csv', index=False)

Way #2: Listing level aggregation . . . can't include reviews  
-Contains listing, number of favorites, description, tags, price

In [63]:
# Merge the listings dataframe with the one that has num of favs per listing
df_favs_for_listings = pd.merge(num_favs_agg, df_listings)

# Add the creation time in epoch seconds
df_favs_for_listings = pd.merge(df_favs_for_listings, listing_tsz_df)

In [64]:
df_favs_for_listings

,listing_id,num_favorites,user_id,title,description,tags,style,is_digital,is_customizable,price,url,creation_tsz
0,262981963,100,38103717,Up inspired our love is sn adventure 3 button,Our LE Up inspired our love is an adventure 3 ...,"['balloons', 'Ellie', 'Carl', 'up', 'disney mo...",NaN,False,False,5.00,https://www.etsy.com/listing/262981963/up-insp...,1539313398
1,591514611,100,103655661,Flower and garden Mickey Mouse ears| flower an...,"Gorgeous rose gold ears with peach, pink and ...","['Mickey ears', 'Mickey Mouse ears', 'Minnie ...",NaN,False,True,35.00,https://www.etsy.com/listing/591514611/flower-...,1533427819
2,528942794,100,30837077,Dcl FE gift (2) LED Lanterns great party favor...,You get 2Stand about 4 inches tall very cute d...,"['Disney shower', 'hidden Mickey', 'DISNEYCRUI...",NaN,False,True,20.00,https://www.etsy.com/listing/528942794/dcl-fe-...,1537277943
3,549566703,100,44053694,Black and White Disney Family Shirts - Disney ...,Halloween Special !! ***********************PL...,"['disney family shirts', 'halloween disney', '...",NaN,False,False,14.00,https://www.etsy.com/listing/549566703/black-a...,1533808614
4,275137002,100,42828296,Color Changing Monorail Shirt in Gray - Limite...,LIMITED QUANTITIES AVAILABLE! This vintage st...,"['Disney World Shirt', 'Magic Kingdom', 'Walt ...",NaN,False,False,20.00,https://www.etsy.com/listing/275137002/color-c...,1537984308
5,226595114,100,33819301,Stormtrooper Magic Band Vinyl Decal,This is a Stormtrooper vinyl decal to apply to...,"['Stormtrooper', 'star wars', 'magic band', 'v...",NaN,False,False,3.00,https://www.etsy.com/listing/226595114/stormtr...,1540317831
6,223503090,100,42828296,EPCOT Shirt - EPCOT Center T-Shirt with Pavili...,"This retro EPCOT shirt reads: Entertain, Infor...","['vintage retro shirt', 'walt disney world', '...",NaN,False,False,25.00,https://www.etsy.com/listing/223503090/epcot-s...,1539572159
7,584325853,100,43098919,Halloween Earrings,These Disney handmade earrings are PERFECT for...,"['Halloween earrings', 'ghost earrings', 'pump...",NaN,False,False,8.00,https://www.etsy.com/listing/584325853/hallowe...,1538875391
8,589443422,100,37407286,"Personalized Shot Glasses, Bridesmaid Shot Gla...",Bottoms up to a great party addition!!!Persona...,"['birthday party', 'bottoms up', 'spring break...",NaN,False,True,3.00,https://www.etsy.com/listing/589443422/persona...,1540169410
9,589646449,100,44053694,Disney Ladies Tank Tops - Disney Rose Gold 201...,Tank tops for the ladies to match their EARS!!...,"['Disney family shirts', 'disney tank tops', '...",NaN,False,False,14.00,https://www.etsy.com/listing/589646449/disney-...,1537579712


In [65]:
# Drop the rows that have shops without Disney-themed items
df_favs_for_listings = df_favs_for_listings[df_favs_for_listings['user_id'] != 37407286]
df_favs_for_listings = df_favs_for_listings[df_favs_for_listings['user_id'] != 82117396]
df_favs_for_listings = df_favs_for_listings[df_favs_for_listings['user_id'] != 120126609]
df_favs_for_listings = df_favs_for_listings[df_favs_for_listings['user_id'] != 21590497]

In [66]:
df_favs_for_listings.shape

(6185, 12)

In [67]:
df_favs_for_listings = df_favs_for_listings.drop_duplicates()

In [68]:
df_favs_for_listings

,listing_id,num_favorites,user_id,title,description,tags,style,is_digital,is_customizable,price,url,creation_tsz
0,262981963,100,38103717,Up inspired our love is sn adventure 3 button,Our LE Up inspired our love is an adventure 3 ...,"['balloons', 'Ellie', 'Carl', 'up', 'disney mo...",NaN,False,False,5.00,https://www.etsy.com/listing/262981963/up-insp...,1539313398
1,591514611,100,103655661,Flower and garden Mickey Mouse ears| flower an...,"Gorgeous rose gold ears with peach, pink and ...","['Mickey ears', 'Mickey Mouse ears', 'Minnie ...",NaN,False,True,35.00,https://www.etsy.com/listing/591514611/flower-...,1533427819
2,528942794,100,30837077,Dcl FE gift (2) LED Lanterns great party favor...,You get 2Stand about 4 inches tall very cute d...,"['Disney shower', 'hidden Mickey', 'DISNEYCRUI...",NaN,False,True,20.00,https://www.etsy.com/listing/528942794/dcl-fe-...,1537277943
3,549566703,100,44053694,Black and White Disney Family Shirts - Disney ...,Halloween Special !! ***********************PL...,"['disney family shirts', 'halloween disney', '...",NaN,False,False,14.00,https://www.etsy.com/listing/549566703/black-a...,1533808614
4,275137002,100,42828296,Color Changing Monorail Shirt in Gray - Limite...,LIMITED QUANTITIES AVAILABLE! This vintage st...,"['Disney World Shirt', 'Magic Kingdom', 'Walt ...",NaN,False,False,20.00,https://www.etsy.com/listing/275137002/color-c...,1537984308
5,226595114,100,33819301,Stormtrooper Magic Band Vinyl Decal,This is a Stormtrooper vinyl decal to apply to...,"['Stormtrooper', 'star wars', 'magic band', 'v...",NaN,False,False,3.00,https://www.etsy.com/listing/226595114/stormtr...,1540317831
6,223503090,100,42828296,EPCOT Shirt - EPCOT Center T-Shirt with Pavili...,"This retro EPCOT shirt reads: Entertain, Infor...","['vintage retro shirt', 'walt disney world', '...",NaN,False,False,25.00,https://www.etsy.com/listing/223503090/epcot-s...,1539572159
7,584325853,100,43098919,Halloween Earrings,These Disney handmade earrings are PERFECT for...,"['Halloween earrings', 'ghost earrings', 'pump...",NaN,False,False,8.00,https://www.etsy.com/listing/584325853/hallowe...,1538875391
9,589646449,100,44053694,Disney Ladies Tank Tops - Disney Rose Gold 201...,Tank tops for the ladies to match their EARS!!...,"['Disney family shirts', 'disney tank tops', '...",NaN,False,False,14.00,https://www.etsy.com/listing/589646449/disney-...,1537579712
10,584104597,100,103655661,Little mermaid Mickey ears | little mermaid ea...,Super cute founder inspired Mickey ears 🐟💛Head...,"['Mickey ears', 'Minnie ears', 'Minnie Mouse e...",NaN,False,True,35.00,https://www.etsy.com/listing/584104597/little-...,1540458937


In [69]:
# Clean it up so the listings are summarized in an easily readable way
df_favs_for_listings = df_favs_for_listings.drop(['url'], axis=1)
# Recode strings to numeric.
df_favs_for_listings['is_digital'] = df_favs_for_listings['is_digital'].map({
                                                                            False: 0, True: 1})
df_favs_for_listings['is_customizable'] = df_favs_for_listings['is_customizable'].map({
                                                                                      False: 0, True: 1})

df_favs_for_listings['style'] = df_favs_for_listings['style'].replace(
    np.nan, '', regex=True)

In [70]:
df_favs_for_listings = df_favs_for_listings.drop_duplicates()
# This is the final listing DF with all the listing data, no duplicates
df_favs_for_listings

,listing_id,num_favorites,user_id,title,description,tags,style,is_digital,is_customizable,price,creation_tsz
0,262981963,100,38103717,Up inspired our love is sn adventure 3 button,Our LE Up inspired our love is an adventure 3 ...,"['balloons', 'Ellie', 'Carl', 'up', 'disney mo...",,0,0,5.00,1539313398
1,591514611,100,103655661,Flower and garden Mickey Mouse ears| flower an...,"Gorgeous rose gold ears with peach, pink and ...","['Mickey ears', 'Mickey Mouse ears', 'Minnie ...",,0,1,35.00,1533427819
2,528942794,100,30837077,Dcl FE gift (2) LED Lanterns great party favor...,You get 2Stand about 4 inches tall very cute d...,"['Disney shower', 'hidden Mickey', 'DISNEYCRUI...",,0,1,20.00,1537277943
3,549566703,100,44053694,Black and White Disney Family Shirts - Disney ...,Halloween Special !! ***********************PL...,"['disney family shirts', 'halloween disney', '...",,0,0,14.00,1533808614
4,275137002,100,42828296,Color Changing Monorail Shirt in Gray - Limite...,LIMITED QUANTITIES AVAILABLE! This vintage st...,"['Disney World Shirt', 'Magic Kingdom', 'Walt ...",,0,0,20.00,1537984308
5,226595114,100,33819301,Stormtrooper Magic Band Vinyl Decal,This is a Stormtrooper vinyl decal to apply to...,"['Stormtrooper', 'star wars', 'magic band', 'v...",,0,0,3.00,1540317831
6,223503090,100,42828296,EPCOT Shirt - EPCOT Center T-Shirt with Pavili...,"This retro EPCOT shirt reads: Entertain, Infor...","['vintage retro shirt', 'walt disney world', '...",,0,0,25.00,1539572159
7,584325853,100,43098919,Halloween Earrings,These Disney handmade earrings are PERFECT for...,"['Halloween earrings', 'ghost earrings', 'pump...",,0,0,8.00,1538875391
9,589646449,100,44053694,Disney Ladies Tank Tops - Disney Rose Gold 201...,Tank tops for the ladies to match their EARS!!...,"['Disney family shirts', 'disney tank tops', '...",,0,0,14.00,1537579712
10,584104597,100,103655661,Little mermaid Mickey ears | little mermaid ea...,Super cute founder inspired Mickey ears 🐟💛Head...,"['Mickey ears', 'Minnie ears', 'Minnie Mouse e...",,0,1,35.00,1540458937


In [71]:
# Add the year of creation for the listings
df_favs_for_listings['creation_tsz'] = pd.to_datetime(
    df_favs_for_listings['creation_tsz'], unit='s')
df_favs_for_listings['creation_tsz'] = df_favs_for_listings['creation_tsz'].dt.year
df_favs_for_listings = df_favs_for_listings.rename(
    columns={'creation_tsz': 'creation_yr'})
df_favs_for_listings

,listing_id,num_favorites,user_id,title,description,tags,style,is_digital,is_customizable,price,creation_yr
0,262981963,100,38103717,Up inspired our love is sn adventure 3 button,Our LE Up inspired our love is an adventure 3 ...,"['balloons', 'Ellie', 'Carl', 'up', 'disney mo...",,0,0,5.00,2018
1,591514611,100,103655661,Flower and garden Mickey Mouse ears| flower an...,"Gorgeous rose gold ears with peach, pink and ...","['Mickey ears', 'Mickey Mouse ears', 'Minnie ...",,0,1,35.00,2018
2,528942794,100,30837077,Dcl FE gift (2) LED Lanterns great party favor...,You get 2Stand about 4 inches tall very cute d...,"['Disney shower', 'hidden Mickey', 'DISNEYCRUI...",,0,1,20.00,2018
3,549566703,100,44053694,Black and White Disney Family Shirts - Disney ...,Halloween Special !! ***********************PL...,"['disney family shirts', 'halloween disney', '...",,0,0,14.00,2018
4,275137002,100,42828296,Color Changing Monorail Shirt in Gray - Limite...,LIMITED QUANTITIES AVAILABLE! This vintage st...,"['Disney World Shirt', 'Magic Kingdom', 'Walt ...",,0,0,20.00,2018
5,226595114,100,33819301,Stormtrooper Magic Band Vinyl Decal,This is a Stormtrooper vinyl decal to apply to...,"['Stormtrooper', 'star wars', 'magic band', 'v...",,0,0,3.00,2018
6,223503090,100,42828296,EPCOT Shirt - EPCOT Center T-Shirt with Pavili...,"This retro EPCOT shirt reads: Entertain, Infor...","['vintage retro shirt', 'walt disney world', '...",,0,0,25.00,2018
7,584325853,100,43098919,Halloween Earrings,These Disney handmade earrings are PERFECT for...,"['Halloween earrings', 'ghost earrings', 'pump...",,0,0,8.00,2018
9,589646449,100,44053694,Disney Ladies Tank Tops - Disney Rose Gold 201...,Tank tops for the ladies to match their EARS!!...,"['Disney family shirts', 'disney tank tops', '...",,0,0,14.00,2018
10,584104597,100,103655661,Little mermaid Mickey ears | little mermaid ea...,Super cute founder inspired Mickey ears 🐟💛Head...,"['Mickey ears', 'Minnie ears', 'Minnie Mouse e...",,0,1,35.00,2018


In [72]:
# Save for future use
df_favs_for_listings.to_csv('df_listing_final.csv')